# Neural Sentiment Analysis using DaCy

DaCy currently does not include its own tools for sentiment extraction, but a couple of good tools already exists. DaCy providers wrappers for these to use them in the spaCy/DaCy framework. This allows you to get all of the best models in one place.

To get started using dacy install it using:

```bash
pip install dacy
```

In [16]:
import dacy
import spacy

# Overview of Models
--- 


| Name  | Creator   | Domain   | Output Type  | Model Type | 
|:---|:-------------|:------|:------|:------|
| `dacy/subjectivity` | [DaNLP](https://github.com/alexandrainst/danlp/blob/master/docs/docs/tasks/sentiment_analysis.md#bert-tone)  | Europarl and Twitter  | `["objective", "subjective"]` | [Danish BERT by BotXO](https://huggingface.co/Maltehb/danish-bert-botxo) | 
| `dacy/polarity` | [DaNLP](https://github.com/alexandrainst/danlp/blob/master/docs/docs/tasks/sentiment_analysis.md#bert-tone)  | Europarl and Twitter  | `['postive', 'neutral', 'negative']` | [Danish BERT by BotXO](https://huggingface.co/Maltehb/danish-bert-botxo) | 
| `dacy/emotion` | [DaNLP](https://github.com/alexandrainst/danlp/blob/master/docs/docs/tasks/sentiment_analysis.md#bert-emotion)  | Social Media  | `["Emotional", "No emotion"] and  ["Glæde/Sindsro", "Tillid/Accept", ... ]` | [Danish BERT by BotXO](https://huggingface.co/Maltehb/danish-bert-botxo) | 
| Asent | [Asent](https://kennethenevoldsen.github.io/asent/index.html)    | Microblogs and Social media  | `Polarity score (continuous)`     | Rule-based | 

> *Note* To see how to use the rule-based model Asent for sentiment analysis check out the respective [guide](https://centre-for-humanities-computing.github.io/DaCy/dacy.sentiment.html).


## Subjectivity
---

The subjectivity model is a part of BertTone a model trained by
[DaNLP](https://github.com/alexandrainst/danlp). The models detect whether a
text is subjective or not.

To add the subjectivity model to your pipeline simply run:

In [17]:
nlp = spacy.blank("da") # an empty spacy pipeline
nlp.add_pipe("dacy/subjectivity")
print("Added subjectivity compoenent")

Added subjectivity compoenent


This will add the `dacy.subjectivity` component to your pipeline, which adds
two extensions to the Doc object,`subjectivity_prob` and `subjectivity`.
These show the probabilities of a document being subjective and whether not a
document is subjective or objective. Let's look at an example using the model:

In [18]:
texts = [
    "Analysen viser, at økonomien bliver forfærdelig dårlig",
    "Jeg tror alligevel, det bliver godt",
]

docs = nlp.pipe(texts)

for doc in docs:
    print(doc._.subjectivity)
    print(doc._.subjectivity_prob)

objective
{'prob': array([1., 0.], dtype=float32), 'labels': ['objective', 'subjective']}
subjective
{'prob': array([0., 1.], dtype=float32), 'labels': ['objective', 'subjective']}


## Polarity
---

Similar to the subjectivity model, the polarity model is a of the BertTone model. 
This model classifies the polarity of a text, i.e. whether it is positve,
negative or neutral.

To add the polarity model to your pipeline simply run:


In [19]:
nlp = spacy.blank("da") # an empty spacy pipeline
nlp.add_pipe("dacy/polarity")
print("Added polarity compoenent")

Added polarity compoenent


This will add the `dacy.polarity` component to your pipeline, which adds
two extensions to the Doc object,`polarity_prob` and `polarity`.
These show the probabilities of a document being positive/neutral/negative and
the resulting classification. Let's look at an example using the model:

In [20]:
# apply the pipeline
docs = nlp.pipe(texts)

for doc in docs:
    # print the model predictions
    print(doc._.polarity)
    print(doc._.polarity_prob)

negative
{'prob': array([0.002, 0.008, 0.99 ], dtype=float32), 'labels': ['positive', 'neutral', 'negative']}
positive
{'prob': array([0.981, 0.019, 0.   ], dtype=float32), 'labels': ['positive', 'neutral', 'negative']}


## Emotion
---

The emotion model used in DaCy is trained by
[DaNLP](https://github.com/alexandrainst/danlp). It exists of two models.
One for detecting wether a text is emotionally laden and one for classifying
which emotion it is out of the following emotions:
Similar to the subjectivity model, the polarity model is a of the BertTone model. 
This model classifies the polarity of a text, i.e. whether it is positve,
negative or neutral.

- "Glæde/Sindsro" (happiness)
- "Tillid/Accept" (trust/acceptance)
- "Forventning/Interrese" (interest)
- "Overasket/Målløs" (surprise)
- "Vrede/Irritation" (Anger)
- "Foragt/Modvilje" (Contempt)
- "Sorg/trist" (Sadness)
- "Frygt/Bekymret" (Fear)

To add the emotion models to your pipeline simply run:


In [21]:
nlp = spacy.blank("da")
nlp.add_pipe("dacy/emotionally_laden")
nlp.add_pipe("dacy/emotion")

print("Pipeline components added")

/Users/au561649/Desktop/Github/DaCy/dacy/sentiment/wrapped_models.py:142: UserWarning: The 'emotion' component assumes the 'emotionally_laden' extension is set. To set it you can run  nlp.add_pipe('dacy.emotionally_laden')
  warn(


Pipeline components added


This wil set the two extensions to the Doc object, `laden` and `emotion`.
These shows whether a text is emotionally laden and what emotion it contains.
Both of these also come with `*_prob`-suffix if you want to examine the
probabilites of the model.

Let's look at an example using the model:

In [22]:
texts = [
    "Ej den bil er såå flot",
    "Fuck det er bare så FUCKING træls!",
    "Har i set at Tesla har landet en raket på månen? Det er vildt!!",
    "der er et træ i haven"
]

docs = nlp.pipe(texts)

for doc in docs:
    print(doc._.emotionally_laden)
    # if emotional print the emotion
    if doc._.emotionally_laden == "emotional":
        print("\t", doc._.emotion)

emotional
	 tillid/accept
emotional
	 sorg/trist
emotional
	 overasket/målløs
no emotion
